In [11]:
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from time import time
from lxml import html,etree
import requests,re
import os,sys
import unicodecsv as csv
import urllib
import time as tm

start_time = tm.time()
localities = ["San Francisco, CA","Washington DC","Philadelphia, PA","Los Angeles, CA","Boston, MA","San Diego, CA"]
for locality in localities:
    checkin_date = datetime.strptime("2017/06/11","%Y/%m/%d")
    checkout_date = datetime.strptime("2017/06/12","%Y/%m/%d")
    sort= "popularity"
    checkIn = checkin_date.strftime("%Y/%m/%d")
    checkOut = checkout_date.strftime("%Y/%m/%d")
    print("Scraper Inititated for Locality:%s"%locality)
    # TA rendering the autocomplete list using this API
    print("Finding search result page URL")

    geo_url = 'https://www.tripadvisor.com/TypeAheadJson?action=API&startTime='+str(int(time()))+'&uiOrigin=GEOSCOPE&source=GEOSCOPE&interleaved=true&types=geo,theme_park&neighborhood_geos=true&link_type=hotel&details=true&max=12&injectNeighborhoods=true&query='+locality
    api_response  = requests.get(geo_url).json()
    #getting the TA url for th equery from the autocomplete response
    url = "http://www.tripadvisor.com"+api_response['results'][0]['url']

    ret = 1
    raw_hotel_link = []
    raw_no_of_reviews = []
    raw_rank = []
    raw_rating = []
    raw_hotel_name = [] 
    raw_booking_provider = []
    hotel_data = dict()

    while ret:
        print('URL found %s'%url)
        html = urllib.request.urlopen(url)
        soup = BeautifulSoup(html,'lxml') 
        data = dict()

        for hlink in soup.find_all("div", {'class':'listing_title'}):
                for al in hlink.find_all("a", {'class':'property_title'}):
                    raw_hotel_link.append('http://www.tripadvisor.com' + al['href'])

        for rvcnt in soup.find_all("span", {'class':'more review_count'}):
            raw_no_of_reviews.append(rvcnt.getText().replace(',',''))    


        for pop in soup.find_all("div", {'class':'popRanking'}):
            raw_rank.append(pop.getText().split(" ")[0])


        for rating in soup.find_all("div", {'class':'rating'}):
            for rt in rating.find_all("div",{'class':'prw_rup prw_common_bubble_rating'}):
                raw_rating.append(rt.find("span")['alt'].split(" ")[0])

        for hname in soup.find_all("a", {'class':'property_title'}):
            raw_hotel_name.append(hname.getText())  


        for img in soup.find_all('div',attrs={"class" : "providerLogo"}):
            for al in img.find_all('img'):
                raw_booking_provider.append(al['alt'])  

        data['hotel_url'] = raw_hotel_link
        data['no_of_reviews'] = raw_no_of_reviews
        data['hotel_rank'] = raw_rank
        data['rating'] = raw_rating
        data['booking_provider'] = raw_booking_provider  
        data['hotel_name'] = raw_hotel_name

        hotel_data.update(data)

        next_page = soup.find('link', rel="next")
        if next_page:
            next_page_url = next_page['href']

        next_url = 'http://www.tripadvisor.com' + next_page_url    

        if next_page:
            url = 'http://www.tripadvisor.com' + next_page_url
        else:
            ret = 0
            
        review_df = pd.DataFrame.from_dict(hotel_data, orient='index')
        review_df = review_df.transpose()
        review_df = review_df[['hotel_name','hotel_url','hotel_rank','rating','booking_provider','no_of_reviews']]
        review_df.to_csv('%s_tripadvisor_data.csv' % locality)          

print("Completed in ", tm.time() - start_time, " seconds")

Scraper Inititated for Locality:San Francisco, CA
Finding search result page URL
URL found http://www.tripadvisor.com/Hotels-g60713-San_Francisco_California-Hotels.html
URL found http://www.tripadvisor.com/Hotels-g60713-oa30-San_Francisco_California-Hotels.html
URL found http://www.tripadvisor.com/Hotels-g60713-oa60-San_Francisco_California-Hotels.html
URL found http://www.tripadvisor.com/Hotels-g60713-oa90-San_Francisco_California-Hotels.html
URL found http://www.tripadvisor.com/Hotels-g60713-oa120-San_Francisco_California-Hotels.html
URL found http://www.tripadvisor.com/Hotels-g60713-oa150-San_Francisco_California-Hotels.html
URL found http://www.tripadvisor.com/Hotels-g60713-oa180-San_Francisco_California-Hotels.html
URL found http://www.tripadvisor.com/Hotels-g60713-oa210-San_Francisco_California-Hotels.html
Scraper Inititated for Locality:Washington DC
Finding search result page URL
URL found http://www.tripadvisor.com/Hotels-g28970-Washington_DC_District_of_Columbia-Hotels.html
U

In [13]:
pd.set_option('display.max_colwidth', -1)

In [48]:
import pandas as pd
import numpy as np
sandf = pd.read_csv("./Area_hotels/San Francisco, CA_tripadvisor_data.csv")
wasntondf = pd.read_csv("./Area_hotels/Washington DC_tripadvisor_data.csv")
phildf = pd.read_csv("./Area_hotels/Philadelphia, PA_tripadvisor_data.csv")
losangldf = pd.read_csv("./Area_hotels/Los Angeles, CA_tripadvisor_data.csv")
bostondf = pd.read_csv("./Area_hotels/Boston, MA_tripadvisor_data.csv")
sandgdf = pd.read_csv("./Area_hotels/San Diego, CA_tripadvisor_data.csv")

Embassy_Suites = phildf[phildf['hotel_name'].str.startswith('Embassy Suites')]['hotel_url'][45] #075
Le_Montrose = losangldf[losangldf['hotel_name'].str.startswith('Le Montrose')]['hotel_url'][335]#.875
Georgetown_Suites = wasntondf[wasntondf['hotel_name'] == 'Georgetown Suites']['hotel_url'][35] #.76
Harbor_Court = sandf[sandf['hotel_name'].str.startswith('Harbor Court')]['hotel_url'][66] #1.17
Hilton = sandgdf[sandgdf['hotel_name'].str.startswith('Hilton San Diego Resort')]['hotel_url'][8]#2.98

Park_Central = sandf[sandf['hotel_name'].str.startswith('Park Central')]['hotel_url'][21] #3.1
WestinCopley = bostondf[bostondf['hotel_name'].str.contains('Westin Copley')]['hotel_url'][23] #3.5

In [74]:
Hilton

'http://www.tripadvisor.com/Hotel_Review-g60750-d119721-Reviews-Hilton_San_Diego_Resort_Spa-San_Diego_California.html'

### Embassy_Suites

In [57]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
import re
import time
from textblob import TextBlob

start_time = time.time()
ret = 1
page_num = 1
rating = []
name = []
review_date = []
title = []
review = []
all_data = dict()
url = Embassy_Suites
driver = webdriver.Chrome()   
    
while ret:
    print(url)
    
    driver.get(url)
#     wait = ui.WebDriverWait(driver,10)
    
    mores = driver.find_elements_by_css_selector('span.taLnk.ulBlueLinks')
    for more in mores:
        driver.execute_script("arguments[0].click()", more) 

    time.sleep(10)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')   
    
    data = dict()

    for mgr in soup.find_all("div", {'class':'mgrRspnInline'}): 
        mgr.decompose()
    for cnt in soup.find_all("div", {'class':'content_block answers_block tabs_answers scroll_tabs'}):
        cnt.decompose()    

    for sp in soup.find_all("div", {'class':'rating reviewItemInline'}):
        sp = str(sp.find("span")['class'][1])[7]
        rating.append(sp)

    for uinfo in soup.find_all("div", {'class':'username mo'}):
        name.append(uinfo.getText())

    for rv in soup.find_all("p", {'class':'partial_entry'}): 
        review.append(rv.getText())

    for tt in soup.find_all("span", {'class':'noQuotes'}):
        title.append(tt.getText())

    for rvdt in soup.find_all("span", {'class':'ratingDate relativeDate'}):
        review_date.append(rvdt['title']) 

    page_number = int(soup.find('span', class_="pageNum current")['data-page-number'])

    data['title'] = title
    data['review'] = review
    data['review_date'] = review_date
    data['rating'] = rating
    data['name'] = name
    

    all_data.update(data)  
    all_data['review'] = [s.strip('\n') for s in all_data['review']]
    all_data['name'] = [s.strip('\n') for s in all_data['name']]
    
    print(page_num)
    page_num += 1
    
    next_page = soup.find('link', rel="next")
    if next_page:
        next_page_url = next_page['href']
        
    next_url = 'http://www.tripadvisor.com' + next_page_url    
    
    if next_page:
        url = 'http://www.tripadvisor.com' + next_page_url
    else:
        ret = 0

review_df = pd.DataFrame.from_dict(all_data, orient='index')
review_df = review_df.transpose()
review_df = review_df[['review_date','name','rating','title','review']]
review_df.to_excel('Embassy_Suites.xlsx')      
       
print("---took %s seconds ---" % (time.time() - start_time))
print("Data Imported")           

http://www.tripadvisor.com/Hotel_Review-g60795-d96688-Reviews-Embassy_Suites_by_Hilton_Philadelphia_Center_City-Philadelphia_Pennsylvania.html
1
http://www.tripadvisor.com/Hotel_Review-g60795-d96688-Reviews-or5-Embassy_Suites_by_Hilton_Philadelphia_Center_City-Philadelphia_Pennsylvania.html
2
http://www.tripadvisor.com/Hotel_Review-g60795-d96688-Reviews-or10-Embassy_Suites_by_Hilton_Philadelphia_Center_City-Philadelphia_Pennsylvania.html
3
http://www.tripadvisor.com/Hotel_Review-g60795-d96688-Reviews-or15-Embassy_Suites_by_Hilton_Philadelphia_Center_City-Philadelphia_Pennsylvania.html
4
http://www.tripadvisor.com/Hotel_Review-g60795-d96688-Reviews-or20-Embassy_Suites_by_Hilton_Philadelphia_Center_City-Philadelphia_Pennsylvania.html
5
http://www.tripadvisor.com/Hotel_Review-g60795-d96688-Reviews-or25-Embassy_Suites_by_Hilton_Philadelphia_Center_City-Philadelphia_Pennsylvania.html
6
http://www.tripadvisor.com/Hotel_Review-g60795-d96688-Reviews-or30-Embassy_Suites_by_Hilton_Philadelphia_C

### Le_Montrose

In [61]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
import re
import time

start_time = time.time()
ret = 1
page_num = 1
rating = []
name = []
review_date = []
title = []
review = []
all_data = dict()
url = Le_Montrose
driver = webdriver.Chrome()   
    
while ret:
    print(url)
    
    driver.get(url)
    wait = ui.WebDriverWait(driver,10)
    
    mores = driver.find_elements_by_css_selector('span.taLnk.ulBlueLinks')
    for more in mores:
        driver.execute_script("arguments[0].click()", more) 

    time.sleep(10)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')   
    
    data = dict()

    for mgr in soup.find_all("div", {'class':'mgrRspnInline'}): 
        mgr.decompose()
    for cnt in soup.find_all("div", {'class':'content_block answers_block tabs_answers scroll_tabs'}):
        cnt.decompose()    

    for sp in soup.find_all("div", {'class':'rating reviewItemInline'}):
        sp = str(sp.find("span")['class'][1])[7]
        rating.append(sp)

    for uinfo in soup.find_all("div", {'class':'username mo'}):
        name.append(uinfo.getText())

    for rv in soup.find_all("p", {'class':'partial_entry'}): 
        review.append(rv.getText())

    for tt in soup.find_all("span", {'class':'noQuotes'}):
        title.append(tt.getText())

    for rvdt in soup.find_all("span", {'class':'ratingDate relativeDate'}):
        review_date.append(rvdt['title']) 

    page_number = int(soup.find('span', class_="pageNum current")['data-page-number'])

    data['review_date'] = review_date
    data['title'] = title
    data['review'] = review
    data['rating'] = rating
    data['name'] = name

    all_data.update(data)  
    all_data['review'] = [s.strip('\n') for s in all_data['review']]
    all_data['name'] = [s.strip('\n') for s in all_data['name']]
    
    print(page_num)
    page_num += 1
    
    next_page = soup.find('link', rel="next")
    if next_page:
        next_page_url = next_page['href']
        
    next_url = 'http://www.tripadvisor.com' + next_page_url    
    
    if next_page:
        url = 'http://www.tripadvisor.com' + next_page_url
    else:
        ret = 0

review_df = pd.DataFrame.from_dict(all_data, orient='index')
review_df = review_df.transpose()
review_df = review_df[['review_date','name','rating','title','review']]
review_df.to_excel('Le_Montrose.xlsx')      
       
print("---took %s seconds ---" % (time.time() - start_time))
print("Data Imported")           

http://www.tripadvisor.com/Hotel_Review-g33252-d82586-Reviews-Le_Montrose_Suite_Hotel-West_Hollywood_California.html
1
http://www.tripadvisor.com/Hotel_Review-g33252-d82586-Reviews-or5-Le_Montrose_Suite_Hotel-West_Hollywood_California.html
2
http://www.tripadvisor.com/Hotel_Review-g33252-d82586-Reviews-or10-Le_Montrose_Suite_Hotel-West_Hollywood_California.html
3
http://www.tripadvisor.com/Hotel_Review-g33252-d82586-Reviews-or15-Le_Montrose_Suite_Hotel-West_Hollywood_California.html
4
http://www.tripadvisor.com/Hotel_Review-g33252-d82586-Reviews-or20-Le_Montrose_Suite_Hotel-West_Hollywood_California.html
5
http://www.tripadvisor.com/Hotel_Review-g33252-d82586-Reviews-or25-Le_Montrose_Suite_Hotel-West_Hollywood_California.html
6
http://www.tripadvisor.com/Hotel_Review-g33252-d82586-Reviews-or30-Le_Montrose_Suite_Hotel-West_Hollywood_California.html
7
http://www.tripadvisor.com/Hotel_Review-g33252-d82586-Reviews-or35-Le_Montrose_Suite_Hotel-West_Hollywood_California.html
8
http://www.tri

### Georgetown_Suites

In [71]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
import time

start_time = time.time()
ret = 1
page_num = 1
rating = []
name = []
review_date = []
title = []
review = []
all_data = dict()
url = Georgetown_Suites
driver = webdriver.Chrome()   
    
while ret:
    print(url)
    
    driver.get(url)
    wait = ui.WebDriverWait(driver,10)
    
    mores = driver.find_elements_by_css_selector('span.taLnk.ulBlueLinks')
    for more in mores:
        driver.execute_script("arguments[0].click()", more) 

    time.sleep(10)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')   
    
    data = dict()

    for mgr in soup.find_all("div", {'class':'mgrRspnInline'}): 
        mgr.decompose()
    for cnt in soup.find_all("div", {'class':'content_block answers_block tabs_answers scroll_tabs'}):
        cnt.decompose()    

    for sp in soup.find_all("div", {'class':'rating reviewItemInline'}):
        sp = str(sp.find("span")['class'][1])[7]
        rating.append(sp)

    for uinfo in soup.find_all("div", {'class':'username mo'}):
        name.append(uinfo.getText())

    for rv in soup.find_all("p", {'class':'partial_entry'}): 
        review.append(rv.getText())

    for tt in soup.find_all("span", {'class':'noQuotes'}):
        title.append(tt.getText())

    for rvdt in soup.find_all("span", {'class':'ratingDate relativeDate'}):
        review_date.append(rvdt['title']) 

    page_number = int(soup.find('span', class_="pageNum current")['data-page-number'])

    data['review_date'] = review_date
    data['title'] = title
    data['review'] = review
    data['rating'] = rating
    data['name'] = name

    all_data.update(data)  
    all_data['review'] = [s.strip('\n') for s in all_data['review']]
    all_data['name'] = [s.strip('\n') for s in all_data['name']]
    
    print(page_num)
    page_num += 1
    
    next_page = soup.find('link', rel="next")
    if next_page:
        next_page_url = next_page['href']
        
    next_url = 'http://www.tripadvisor.com' + next_page_url    
    
    if next_page:
        url = 'http://www.tripadvisor.com' + next_page_url
    else:
        ret = 0
        
review_df = pd.DataFrame.from_dict(all_data, orient='index')
review_df = review_df.transpose()
review_df = review_df[['review_date','name','rating','title','review']]
review_df.to_excel('./hotel_review/Georgetown_Suites.xlsx')
       
print("---took %s seconds ---" % (time.time() - start_time))
print("Data Imported")           

http://www.tripadvisor.com/Hotel_Review-g28970-d240271-Reviews-Georgetown_Suites-Washington_DC_District_of_Columbia.html
1
http://www.tripadvisor.com/Hotel_Review-g28970-d240271-Reviews-or5-Georgetown_Suites-Washington_DC_District_of_Columbia.html
2
http://www.tripadvisor.com/Hotel_Review-g28970-d240271-Reviews-or10-Georgetown_Suites-Washington_DC_District_of_Columbia.html
3
http://www.tripadvisor.com/Hotel_Review-g28970-d240271-Reviews-or15-Georgetown_Suites-Washington_DC_District_of_Columbia.html
4
http://www.tripadvisor.com/Hotel_Review-g28970-d240271-Reviews-or20-Georgetown_Suites-Washington_DC_District_of_Columbia.html
5
http://www.tripadvisor.com/Hotel_Review-g28970-d240271-Reviews-or25-Georgetown_Suites-Washington_DC_District_of_Columbia.html
6
http://www.tripadvisor.com/Hotel_Review-g28970-d240271-Reviews-or30-Georgetown_Suites-Washington_DC_District_of_Columbia.html
7
http://www.tripadvisor.com/Hotel_Review-g28970-d240271-Reviews-or35-Georgetown_Suites-Washington_DC_District_o

### Harbor_Court

In [73]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
import time

start_time = time.time()
ret = 1
page_num = 1
rating = []
name = []
review_date = []
title = []
review = []
all_data = dict()
url = Harbor_Court
driver = webdriver.Chrome()   
    
while ret:
    print(url)
    
    driver.get(url)
    wait = ui.WebDriverWait(driver,10)
    
    mores = driver.find_elements_by_css_selector('span.taLnk.ulBlueLinks')
    for more in mores:
        driver.execute_script("arguments[0].click()", more) 

    time.sleep(10)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')   
    
    data = dict()

    for mgr in soup.find_all("div", {'class':'mgrRspnInline'}): 
        mgr.decompose()
    for cnt in soup.find_all("div", {'class':'content_block answers_block tabs_answers scroll_tabs'}):
        cnt.decompose()    

    for sp in soup.find_all("div", {'class':'rating reviewItemInline'}):
        sp = str(sp.find("span")['class'][1])[7]
        rating.append(sp)

    for uinfo in soup.find_all("div", {'class':'username mo'}):
        name.append(uinfo.getText())

    for rv in soup.find_all("p", {'class':'partial_entry'}): 
        review.append(rv.getText())

    for tt in soup.find_all("span", {'class':'noQuotes'}):
        title.append(tt.getText())

    for rvdt in soup.find_all("span", {'class':'ratingDate relativeDate'}):
        review_date.append(rvdt['title']) 

    page_number = int(soup.find('span', class_="pageNum current")['data-page-number'])

    data['review_date'] = review_date
    data['title'] = title
    data['review'] = review
    data['rating'] = rating
    data['name'] = name

    all_data.update(data)  
    all_data['review'] = [s.strip('\n') for s in all_data['review']]
    all_data['name'] = [s.strip('\n') for s in all_data['name']]
    
    print(page_num)
    page_num += 1
    
    next_page = soup.find('link', rel="next")
    if next_page:
        next_page_url = next_page['href']
        
    next_url = 'http://www.tripadvisor.com' + next_page_url    
    
    if next_page:
        url = 'http://www.tripadvisor.com' + next_page_url
    else:
        ret = 0
        
review_df = pd.DataFrame.from_dict(all_data, orient='index')
review_df = review_df.transpose()
review_df = review_df[['review_date','name','rating','title','review']]
review_df.to_excel('./hotel_review/Harbor_Court.xlsx')
       
print("---took %s seconds ---" % (time.time() - start_time))
print("Data Imported")           

http://www.tripadvisor.com/Hotel_Review-g60713-d81022-Reviews-Harbor_Court_Hotel-San_Francisco_California.html
1
http://www.tripadvisor.com/Hotel_Review-g60713-d81022-Reviews-or5-Harbor_Court_Hotel-San_Francisco_California.html
2
http://www.tripadvisor.com/Hotel_Review-g60713-d81022-Reviews-or10-Harbor_Court_Hotel-San_Francisco_California.html
3
http://www.tripadvisor.com/Hotel_Review-g60713-d81022-Reviews-or15-Harbor_Court_Hotel-San_Francisco_California.html
4
http://www.tripadvisor.com/Hotel_Review-g60713-d81022-Reviews-or20-Harbor_Court_Hotel-San_Francisco_California.html
5
http://www.tripadvisor.com/Hotel_Review-g60713-d81022-Reviews-or25-Harbor_Court_Hotel-San_Francisco_California.html
6
http://www.tripadvisor.com/Hotel_Review-g60713-d81022-Reviews-or30-Harbor_Court_Hotel-San_Francisco_California.html
7
http://www.tripadvisor.com/Hotel_Review-g60713-d81022-Reviews-or35-Harbor_Court_Hotel-San_Francisco_California.html
8
http://www.tripadvisor.com/Hotel_Review-g60713-d81022-Reviews-

### Hilton

In [80]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
import time

start_time = time.time()
ret = 1
page_num = 1
rating = []
name = []
review_date = []
title = []
review = []
all_data = dict()
url = Hilton
driver = webdriver.Chrome()   
    
while ret:
    print(url)
    
    driver.get(url)
    wait = ui.WebDriverWait(driver,10)
    
    mores = driver.find_elements_by_css_selector('span.taLnk.ulBlueLinks')
    for more in mores:
        driver.execute_script("arguments[0].click()", more) 

    time.sleep(10)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')   
    
    data = dict()

    for mgr in soup.find_all("div", {'class':'mgrRspnInline'}): 
        mgr.decompose()
    for cnt in soup.find_all("div", {'class':'content_block answers_block tabs_answers scroll_tabs'}):
        cnt.decompose()    

    for sp in soup.find_all("div", {'class':'rating reviewItemInline'}):
        sp = str(sp.find("span")['class'][1])[7]
        rating.append(sp)

    for uinfo in soup.find_all("div", {'class':'username mo'}):
        name.append(uinfo.getText())

    for rv in soup.find_all("p", {'class':'partial_entry'}): 
        review.append(rv.getText())

    for tt in soup.find_all("span", {'class':'noQuotes'}):
        title.append(tt.getText())

    for rvdt in soup.find_all("span", {'class':'ratingDate relativeDate'}):
        review_date.append(rvdt['title']) 

    page_number = int(soup.find('span', class_="pageNum current")['data-page-number'])

    data['review_date'] = review_date
    data['title'] = title
    data['review'] = review
    data['rating'] = rating
    data['name'] = name

    all_data.update(data)  
    all_data['review'] = [s.strip('\n') for s in all_data['review']]
    all_data['name'] = [s.strip('\n') for s in all_data['name']]
    
    print(page_num)
    page_num += 1
    
    next_page = soup.find('link', rel="next")
    if next_page:
        next_page_url = next_page['href']
        
    next_url = 'http://www.tripadvisor.com' + next_page_url    
    
    if next_page:
        url = 'http://www.tripadvisor.com' + next_page_url
    else:
        ret = 0
        
review_df = pd.DataFrame.from_dict(all_data, orient='index')
review_df = review_df.transpose()
review_df = review_df[['review_date','name','rating','title','review']]
review_df.to_excel('./hotel_review/Hilton.xlsx')
       
print("---took %s seconds ---" % (time.time() - start_time))
print("Data Imported")           

https://www.tripadvisor.in/Hotel_Review-g60750-d119721-Reviews-or2140-Hilton_San_Diego_Resort_Spa-San_Diego_California.html
1
http://www.tripadvisor.com/Hotel_Review-g60750-d119721-Reviews-or2145-Hilton_San_Diego_Resort_Spa-San_Diego_California.html
2
http://www.tripadvisor.com/Hotel_Review-g60750-d119721-Reviews-or2150-Hilton_San_Diego_Resort_Spa-San_Diego_California.html
3
http://www.tripadvisor.com/Hotel_Review-g60750-d119721-Reviews-or2155-Hilton_San_Diego_Resort_Spa-San_Diego_California.html
4
http://www.tripadvisor.com/Hotel_Review-g60750-d119721-Reviews-or2160-Hilton_San_Diego_Resort_Spa-San_Diego_California.html
5
http://www.tripadvisor.com/Hotel_Review-g60750-d119721-Reviews-or2165-Hilton_San_Diego_Resort_Spa-San_Diego_California.html
6
http://www.tripadvisor.com/Hotel_Review-g60750-d119721-Reviews-or2170-Hilton_San_Diego_Resort_Spa-San_Diego_California.html
7
http://www.tripadvisor.com/Hotel_Review-g60750-d119721-Reviews-or2175-Hilton_San_Diego_Resort_Spa-San_Diego_California